# Data Cleaning

In [2]:
import pandas as pd
import numpy as np

In [3]:
with open("raw/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv","r") as f:
    df = pd.read_csv(f)
df.head()

/var/folders/wc/9wclntk940q3y52dpvt1g_s80000gn/T/ipykernel_11000/3920661926.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


,transit_timestamp,station_complex_id,station_complex,borough,routes,payment_method,ridership,transfers,latitude,longitude,Georeference,itsuid
0,10/14/2022 05:00:00 AM,H007,1 Av (L),M,L,omny,31,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2022-10-14T05:00:00H0071 Av (L)MLomny
1,11/05/2022 03:00:00 AM,H007,1 Av (L),M,L,omny,78,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2022-11-05T03:00:00H0071 Av (L)MLomny
2,01/14/2023 03:00:00 PM,H007,1 Av (L),M,L,omny,14,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2023-01-14T15:00:00H0071 Av (L)MLomny
3,03/23/2023 03:00:00 AM,H007,1 Av (L),M,L,omny,10,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2023-03-23T03:00:00H0071 Av (L)MLomny
4,06/24/2023 06:00:00 AM,H007,1 Av (L),M,L,omny,44,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2023-06-24T06:00:00H0071 Av (L)MLomny


In [30]:
subset_date = "10/28/2023"
subset = df[df["transit_timestamp"].str.contains(subset_date)].reset_index(drop=True)
subset = subset.drop(columns = ["transit_timestamp","Georeference","itsuid","station_complex"])
subset.head()

,station_complex_id,borough,routes,payment_method,ridership,transfers,latitude,longitude
0,H007,M,L,omny,690,11,40.730953,-73.981628
1,H007,M,L,metrocard,165,4,40.730953,-73.981628
2,H007,M,L,metrocard,87,1,40.730953,-73.981628
3,H007,M,L,omny,503,4,40.730953,-73.981628
4,H007,M,L,metrocard,74,0,40.730953,-73.981628


In [31]:
subset["ridership"] = pd.to_numeric(subset["ridership"])
subset["transfers"] = pd.to_numeric(subset["transfers"])
subset["latitude"] = pd.to_numeric(subset["latitude"])
subset["longitude"] = pd.to_numeric(subset["longitude"])
subset = subset.groupby([c for c in subset.columns if c not in ["ridership","transfers"]]).agg({c:"sum" for c in ["ridership","transfers"]}).reset_index()

In [32]:
subset["routes"] = subset["routes"].astype(str)
subset["routes"] = subset["routes"].apply(lambda x:  [item.strip() for item in x.split(',') if item.strip()])

In [33]:
subset.head()

,station_complex_id,borough,routes,payment_method,latitude,longitude,ridership,transfers
0,A002,M,"[5, N, R, 4, W, 6]",metrocard,40.762661,-73.967255,11868,995
1,A002,M,"[5, N, R, 4, W, 6]",omny,40.762661,-73.967255,15570,2348
2,A006,M,"[N, R, W]",metrocard,40.764812,-73.973351,5546,71
3,A006,M,"[N, R, W]",omny,40.764812,-73.973351,7802,238
4,A010,M,"[N, R, Q, W]",metrocard,40.764664,-73.980659,9293,153


In [34]:
med_lat = np.median(list(subset["latitude"].unique()))
med_long = np.median(list(subset["longitude"].unique()))

def assign_region(long, lat):
    region = "S" if lat <= med_lat else "N"
    region += "W" if long <= med_long else "E"
    return region

def assign_borough(borough):
    boroughs = {
        'BK': 'Brooklyn',
        'BX': 'Bronx',
        'M': 'Manhattan',
        'Q': 'Queens',
        'SI': 'Staten Island'
    }
    return boroughs[borough]

def get_percap(borough, val):
    borough_population = {
        "Bronx":1379946,
        "Brooklyn":2590516,
        "Manhattan":1596273,
        "Queens":2278029,
        "Staten Island":491133
    }
    return val/borough_population[borough]

subset["region"] = subset.apply(lambda row: assign_region(row["longitude"], row["latitude"]), axis=1)
subset["borough"] = subset["borough"].apply(lambda b: assign_borough(b))
subset["ridership_percap"] = subset.apply(lambda row: get_percap(row["borough"], row["ridership"]), axis=1)
subset["transfers_percap"] = subset.apply(lambda row: get_percap(row["borough"], row["transfers"]), axis=1)

subset.head()

,station_complex_id,borough,routes,payment_method,latitude,longitude,ridership,transfers,region,ridership_percap,transfers_percap
0,A002,Manhattan,"[5, N, R, 4, W, 6]",metrocard,40.762661,-73.967255,11868,995,NW,0.007435,0.000623
1,A002,Manhattan,"[5, N, R, 4, W, 6]",omny,40.762661,-73.967255,15570,2348,NW,0.009754,0.001471
2,A006,Manhattan,"[N, R, W]",metrocard,40.764812,-73.973351,5546,71,NW,0.003474,0.000044
3,A006,Manhattan,"[N, R, W]",omny,40.764812,-73.973351,7802,238,NW,0.004888,0.000149
4,A010,Manhattan,"[N, R, Q, W]",metrocard,40.764664,-73.980659,9293,153,NW,0.005822,0.000096


In [35]:
with open(f"cleaned/cleaned_{subset_date.replace('/','_')}.csv", "w+") as o:
    subset.to_csv(o, index=False)